In [1]:
!pip install langchain openai duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 16.7 MB/s eta 0:00:00


In [2]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


In [3]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.agents import Tool
from duckduckgo_search import DDGS
import time

In [4]:
# --- SETUP OPENAI KEY ---
os.environ["OPENAI_API_KEY"] = ""  # Replace with your real API key

In [5]:

# --- Define the Tools ---

def web_search(query: str) -> str:
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=2)
        return "\n".join([r["body"] for r in results])

def calculator(expression: str) -> str:
    try:
        result = eval(expression, {"__builtins__": {}})
        return str(result)
    except Exception as e:
        return f"Error calculating: {e}"

# -- Tools List --
tools = {
    "search": web_search,
    "calculate": calculator
}

# --- Define the LLM ---
llm = ChatOpenAI(
    temperature=0.3,
    model_name="gpt-3.5-turbo"
)

# --- Memory ---
memory = []

# --- Core Functions ---

def create_subtasks(goal: str) -> list:
    """Use LLM to break down the big goal into small tasks."""
    prompt = f"Break down the following goal into 3 to 5 clear subtasks:\nGoal: {goal}"
    subtasks = llm.predict(prompt)
    return [task.strip("- ").strip() for task in subtasks.split("\n") if task.strip()]

def decide_tool(task: str) -> str:
    """Simple tool selector based on task."""
    math_keywords = ['+', '-', '*', '/', 'plus', 'minus', 'divide', 'multiply', '**']
    if any(word in task.lower() for word in math_keywords):
        return "calculate"
    else:
        return "search"

def execute_task(task: str, tool_name: str) -> str:
    """Execute the selected tool."""
    tool_func = tools.get(tool_name)
    if tool_func:
        return tool_func(task)
    else:
        return "No appropriate tool found."

def reflect_and_improve():
    """Basic Reflection on what was done."""
    return "I have reflected on past actions and learned better task execution sequencing."

# --- Main Agent Loop ---

def autonomous_agent(goal: str, max_cycles=5):
    print("\n🚀 Starting Autonomous Agent for Goal:", goal)
    steps_completed = 0

    # Step 1: Break Goal into Subtasks
    subtasks = create_subtasks(goal)
    print("\n📋 Planned Subtasks:")
    for idx, subtask in enumerate(subtasks):
        print(f"{idx+1}. {subtask}")

    # Step 2: Execute Subtasks
    for subtask in subtasks:
        if steps_completed >= max_cycles:
            print("\n⏰ Max cycles reached, stopping.")
            break

        print(f"\n🔎 Working on Subtask: {subtask}")

        tool_name = decide_tool(subtask)
        print(f"🛠 Using Tool: {tool_name}")

        result = execute_task(subtask, tool_name)
        print(f"✅ Result: {result}")

        # Save memory
        memory.append({"subtask": subtask, "tool": tool_name, "result": result})
        steps_completed += 1

        # Simulate thinking time
        time.sleep(1)

    # Step 3: Reflection
    print("\n🧠 Reflecting on session...")
    reflection = reflect_and_improve()
    memory.append({"reflection": reflection})

    # Step 4: Summary
    print("\n📚 Final Summary of Actions:")
    for m in memory:
        print(m)

<ipython-input-5-89a70addc3e3>:22: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [6]:
# --- Run Agent ---
if __name__ == "__main__":
    user_goal = input("\n🎯 Enter your BIG GOAL: ")
    autonomous_agent(user_goal)


🎯 Enter your BIG GOAL: Find top AI research papers in 2024 and calculate 23*56

🚀 Starting Autonomous Agent for Goal: Find top AI research papers in 2024 and calculate 23*56


<ipython-input-5-89a70addc3e3>:35: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  subtasks = llm.predict(prompt)



📋 Planned Subtasks:
1. 1. Research and identify top AI research papers published in 2024.
2. 2. Compile a list of the selected research papers and gather relevant information about each paper.
3. 3. Analyze the impact and significance of each research paper in the field of AI.
4. 4. Calculate the result of 23*56 to determine the answer.

🔎 Working on Subtask: 1. Research and identify top AI research papers published in 2024.
🛠 Using Tool: search
✅ Result: Originally published on Towards AI. The Top 10 AI Research Papers of 2024: Key Takeaways and How You Can Apply Them Photo by Maxim Tolchinskiy on Unsplash. As the curtains draw on 2024, it's time to reflect on the innovations that have defined the year in AI. And let's be real — what a year it has been!
Let's dive into the top 10 most cited AI research papers of 2024, ... Published: March 2024. Google's Gemini 1.5 stunned the industry with a 10-million-token context length and rapid ...

🔎 Working on Subtask: 2. Compile a list of the